## Pandas Homework: Heroes of Pymoli Report
### 03/20/2018

In [2]:
# Import packages
import numpy as np
import pandas as pd

# Read in JSON file and create dataframe
file = "purchase_data.json"
df = pd.read_json(file)

# Confirm type
type(df)

# Export to excel
# df.to_excel('pymoli_data.xlsx')

# Inspect dataframe
# df.info
df.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


### 1. Player Count

In [75]:
# 1.1 Count unique number of players
tp = len(df['SN'].unique())

# Convert to Series
s = pd.Series(tp, name='Total Players')
s

# Print solution
df1 = pd.DataFrame(s)
df1

,Total Players
0,573


### 2. Purchasing Analysis (Total)

In [76]:
# 2.1 Count unique number of items
unique = len(df['Item ID'].unique())
col1 = pd.Series(unique, name="Number of Unique Items")

# 2.2 Average Purchase Price
price_avg_tot = round(np.mean(df['Price']),2)
col2 = pd.Series(price_avg_tot, name="Average Purchase Price")

# 2.3 Total Number of Purchases-----------------------------------------------------------------------------
purch_tot = df['Item ID'].count()
col3 = pd.Series(purch_tot, name="Total Number of Purchases")

# 2.4 Total Revenue
rev_tot = np.sum(df['Price'])
col4 = pd.Series(rev_tot, name="Total Revenue")

# Join all in a single dataframe
df2 = pd.DataFrame(col1)
df2 = df2.join(col2)
df2 = df2.join(col3)
df2 = df2.join(col4)

# Print solution
df2

,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,183,2.93,780,2286.33


In [77]:
# 3-- Gender Demographics

df3 = df.groupby('SN')

# 3.1 Filter unique screen names in df2
df_gen = df3['Gender'].unique()

# 3.2 Count totals by gender
gen_ct = df_gen.value_counts()
gen_ct

# 3.3 Calculate percentage by gender
gen_pct = round(gen_ct / unique, 2)
gen_pct

# Join together in dataframe
df3 = pd.DataFrame({"Percentage" : gen_pct,
                      "Total Count" : gen_ct})

# Print solution
df3

,Percentage,Total Count
[Male],2.54,465
[Female],0.55,100
[Other / Non-Disclosed],0.04,8


### 4. Purchasing Analysis (Gender)

In [69]:
# Group by gender and count purchases
gp_pur = df.groupby('Gender').count()
gp_pur

# Create list for Purchase Count
ls_pur = gp_pur['Price']
ls_pur

# Group by gender and find average
gp_avg = df.groupby('Gender').mean()
gp_avg

# Create list for Average Price
ls_avg = round(gp_avg['Price'],2)
ls_avg

# Group by gender and find Total Value
gp_sum = df.groupby('Gender').sum()
gp_sum

# Create list
ls_sum = gp_sum['Price']
ls_sum

# Convert unique count by gender to Series
unique = pd.Series(gen_ct, name = "Total People")
unique

un_count = pd.Series([100, 465, 8])
nf = round(382.91/100, 2)
nm = round(1867.68/465, 2)
no = round(35.74/8, 2)

norm = [nf, nm, no]
normalized = pd.Series(norm, name = "Normalized Totals")

df4 = pd.DataFrame(np.column_stack([ls_pur, ls_avg, ls_sum, normalized]), 
                               columns=['Purchase Count', 'Average Purchase Price', 'Total Purchase Value', 'Normalized Totals'], 
                                 index=['Female', 'Male', 'Other / Non-Disclosed'])

# Print solution
df4

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Female,136.0,2.82,382.91,3.83
Male,633.0,2.95,1867.68,4.02
Other / Non-Disclosed,11.0,3.25,35.74,4.47


### 5. Age Demographics

In [70]:
df['Age'].agg([min, max])

# Create the bins in which Data will be held
bins = [0,10,14,19,24,29,34,39,44,49]

# Create the names for the bins
bin_labels = ["<10", "10-14", "15-19", "20-24","25-29", "30-34","35-39", "40-44", "45-49"]

# Insert new column
df["Age Groups"] = pd.cut(df["Age"], bins, labels=bin_labels)
df.head()

# Calculate
count5 = df.groupby(['Age Groups']).count()["Price"]
avg5 = round(df.groupby(['Age Groups']).mean()["Price"],2)
sum5 = df.groupby(['Age Groups']).sum()["Price"]
norm5 = round(sum5 / count5, 2)

# Add to dataframe
df50 = pd.DataFrame({"Purchase Count":count5, "Average Purchase Price": avg5, 
                     "Total Purchase Value": sum5, "Normalized Totals": norm5})
# Print solution
df50

,Average Purchase Price,Normalized Totals,Purchase Count,Total Purchase Value
Age Groups,,,,
<10,3.02,3.02,32,96.62
10-14,2.70,2.70,31,83.79
15-19,2.91,2.91,133,386.42
20-24,2.91,2.91,336,978.77
25-29,2.96,2.96,125,370.33
30-34,3.08,3.08,64,197.25
35-39,2.84,2.84,42,119.40
40-44,3.19,3.19,16,51.03
45-49,2.72,2.72,1,2.72


### 6. Top Spenders

    # 5 top spenders in the game by total purchase value (group by 'SN', then sum 'Price')
    # list SN, Purchase Count (count of 'price'), Average Purchase Price (avg 'price'), Total Purchase Value (sum 'price')

In [48]:
# Calculate
df6 = df.groupby(['SN']).sum()["Price"]
count = df.groupby(['SN']).count()["Item Name"]
avg = round(df.groupby(['SN']).mean()["Price"],2)

# Add to dataframe
df60 = pd.DataFrame({"Total Purchase Value":df6, "Purchase Count": count, "Average Purchase Price": avg})

# Sort by Total Purchase Value
df60.sort_values("Total Purchase Value", ascending=False).head()

,Average Purchase Price,Purchase Count,Total Purchase Value
SN,,,
Undirrala66,3.41,5,17.06
Saedue76,3.39,4,13.56
Mindimnya67,3.18,4,12.74
Haellysu29,4.24,3,12.73
Eoda93,3.86,3,11.58


### 7. Most Popular Items

    # 5 most popular items by purchase count (groupby item id, then count 'price')
    # list Item ID, Item Name, Purchase Count [count of 'price'], Item Price, Total Purchase Value [sum of 'price']

In [71]:
# Calculate
tot_items = df.groupby(["Item ID", "Item Name"]).count()["Price"]
df_item_ct = df.groupby(["Item ID", "Item Name"]).count()["Price"]
df_item_rev = df.groupby(["Item ID", "Item Name"]).sum()["Price"]

# Add to dataframe
df8 = pd.DataFrame({"Purchase Count":tot_items, "Total Purchase Value":df_item_rev})

# Sort by Purchase Count
df8.sort_values("Purchase Count", ascending=False).head()

,,Purchase Count,Total Purchase Value
Item ID,Item Name,,
39,"Betrayal, Whisper of Grieving Widows",11,25.85
84,Arcane Gem,11,24.53
31,Trickster,9,18.63
175,Woeful Adamantite Claymore,9,11.16
13,Serenity,9,13.41


### 8. Most Profitable Items

    # 5 most profitable items by total purchase value (groupby item id, then sum 'price')
    # list Item ID, Item Name, Purchase Count [count of 'price'], Item Price, Total Purchase Value [sum of 'price']

In [72]:
# Calculate
df_item = df.groupby(["Item ID", "Item Name"]).sum()["Price"]
df_item_ct = df.groupby(["Item ID", "Item Name"]).count()["Price"]
df_item_rev = df.groupby(["Item ID", "Item Name"]).sum()["Price"]

# Add to dataframe
df8 = pd.DataFrame({"Purchase Count":df_item_ct, "Total Purchase Value":df_item_rev})

# Sort by Total Purchase Value
df8.sort_values("Total Purchase Value", ascending=False).head()

,,Purchase Count,Total Purchase Value
Item ID,Item Name,,
34,Retribution Axe,9,37.26
115,Spectral Diamond Doomblade,7,29.75
32,Orenmir,6,29.70
103,Singed Scalpel,6,29.22
107,"Splitter, Foe Of Subtlety",8,28.88


### FINAL CONSIDERATIONS

    # Export markdown version of notebook called README.md

### 3 observable trends

    # 20-24 age group represents the highest sales volume and revenue.
    # The average spend per purchase and normalized totals do not vary significantly between age or gender demographics.
    # Perhaps, not surprisingly, males spent more than 4.5 times females on purchases.